In [13]:
import pandas as pd
import pickle
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta


from tqdm.notebook import tqdm
tqdm.pandas()
from utils import full_clean, create_subframe, create_t3_df, create_t1_df, get_parallel_edges, create_complete_df, merge_df,\
    get_parallel_edges, get_weighted_edges, get_edge_info, create_post_comment_name_id, create_t3_comment_id_dict, total_post_dict
from utils.clean_df import advanced_clean

-----
Dataframes for clean BOT

In [14]:
bot_list = pd.read_csv('/Users/victor/Desktop/CS/UST_summer/data/dataframe/bot_list.csv')
bot_list['Username'] = bot_list.Username.apply(lambda x: x[3:])

In [15]:
popular_author = pd.read_csv('/Users/victor/Desktop/CS/UST_summer/data/dataframe/10k_sub_final.csv')
popular_bot = popular_author[popular_author.is_bot == 1]

-----
load the post and comment dataframe

In [16]:
comment_fp = '/Users/victor/Desktop/CS/UST_summer/data/dataframe/01-20_comments_clean.csv'
comment_df = pd.read_csv(comment_fp, low_memory = False)[['author', 'id', 'created_utc', 'link_id', 'parent_id','clean_body']]

In [17]:
comment_bot, comment_df = full_clean(comment_df, bot_list, popular_bot)

  0%|          | 0/452604 [00:00<?, ?it/s]

create the comment id dictionary

In [18]:
comment_id, level_1_comments = create_t3_comment_id_dict(comment_df)

In [19]:
level_1_comments

,author,id,created_utc,link_id,parent_id,clean_body,is_bot,all_comments
0,optionseller,fconr5l,1577836849,t3_ei96zu,t3_ei96zu,amazing,False,amazingI not in this screen shot and I dont li...
1,Wolf_Of_1337_Street,fconrdc,1577836853,t3_eiatp9,t3_eiatp9,Also nice DD btw Thanks for sharing,False,Also nice DD btw Thanks for sharingIf you actu...
2,BOOGNISH_IS_REAL,fconudk,1577836914,t3_eial7r,t3_eial7r,looks garbo,False,looks garboThey lost 89 5 million last quarter...
3,DemotivatedTurtle,fconuy0,1577836925,t3_ei8tl1,t3_ei8tl1,I've got multiple SPY calls at 314 317 and 320...,False,I've got multiple SPY calls at 314 317 and 320...
4,Juicymess,fconvmp,1577836939,t3_eiaz8e,t3_eiaz8e,I feel like this is supposed to be funny,False,I feel like this is supposed to be funnyWhy is...
...,...,...,...,...,...,...,...,...
160723,npdaly,fg508r4,1580515133,t3_ewuvf8,t3_ewuvf8,Making steamed pork dumplings tonight and drin...,False,GBoom time for the weekendfNew and gayflairfas...
160724,Ardesic53,fg509pz,1580515150,t3_ewuvf8,t3_ewuvf8,Questions for the guys across the pond from th...,False,GBoom time for the weekendfNew and gayflairfas...
160725,cloud_mode,fg50aka,1580515164,t3_ewuvf8,t3_ewuvf8,Good news s Beer virus has been upgraded to H ...,False,GBoom time for the weekendfNew and gayflairfas...
160726,iwop,fg50bry,1580515186,t3_ewx5wv,t3_ewx5wv,i bet they're nice and tendy,False,Well if you made money on tesla you're clearly...


create total post dictionary

In [20]:
post_fp = '/Users/victor/Desktop/CS/UST_summer/data/dataframe/posts_clean_final.csv'
post_df = pd.read_csv(post_fp, low_memory = False)[['id', 'author', 'created_utc', 'clean_title', 'clean_selftext']]

In [21]:
post_df

,id,author,created_utc,clean_title,clean_selftext
0,mhj2yd,USWD_Bank,1.617235e+09,NFT Collectable USWD United States Web Dollar,removed
1,mhj3ew,KingMacias1,1.617235e+09,We made it to Vice News you dirty apes Any pub...,NaN
2,mhj3oa,jt989898,1.617235e+09,Polygon,NaN
3,mhj3vq,KenAdamsonPrime,1.617235e+09,Ubiquiti UI gets PWND by hackers stock goes up,removed
4,mhj4o5,lexmark0331,1.617235e+09,HOFV mega squeeze,removed
...,...,...,...,...,...
1085583,ewxajz,marioz64,1.580514e+09,Man stays at home because he wasn't ill enough...,NaN
1085584,ewxctl,Compher,1.580515e+09,Found this sub and enabled options yesterday n...,NaN
1085585,ewxei7,jyuliahn1,1.580515e+09,Bears be like,removed
1085586,ewxevz,jyuliahn1,1.580515e+09,Bears be like,NaN


In [22]:
total_post_dict = total_post_dict(post_df, comment_id)